In [1]:
from eda_imports import *

import geopandas
import dateparser

seaborn not found
sqlalchemy not found


In [2]:
%time htmls = list(sorted(glob.glob('./data/raw/by_neighborhood/*/201[7-9]/*/*.html')))

CPU times: user 855 ms, sys: 730 ms, total: 1.58 s
Wall time: 9.18 s


In [3]:
len(htmls)

42432

In [4]:
res = []
for f in tqdm(htmls):
    with open(f, 'rt') as opf:
        try:
            doc = opf.read()
            search = re.search(r'var crimeFeatures = (?P<json_str>[^\;]+)', doc)
            data = eval(search.group('json_str'))
            _df = geopandas.GeoDataFrame.from_features(data['features'])
            res.append(_df)
        except Exception as err:
            print(f)

 32%|███▏      | 13406/42432 [00:33<00:57, 501.00it/s]

./data/raw/by_neighborhood/gramercy-park/2018/27/27.html


 42%|████▏     | 17839/42432 [00:44<01:18, 312.11it/s]

./data/raw/by_neighborhood/la-canada-flintridge/2017/15/15.html


 61%|██████    | 25813/42432 [01:02<01:05, 252.51it/s]

...

 79%|███████▉  | 33450/42432 [01:18<00:23, 385.33it/s]

./data/raw/by_neighborhood/southeast-antelope-valley/2017/28/28.html


 99%|█████████▉| 42050/42432 [01:39<00:00, 402.44it/s]

./data/raw/by_neighborhood/windsor-square/2017/37/37.html


100%|██████████| 42432/42432 [01:40<00:00, 421.46it/s]


In [5]:
adf = pd.concat(res)

In [6]:
adf.rename(columns={'datetime': 'datetime_str'}, inplace=True)

In [ ]:
%time
dts = []
for i in tqdm(adf.datetime_str.values):
    dts.append(dateparser.parse(i))

  0%|          | 16/442383 [00:00<46:15, 159.37it/s]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


 53%|█████▎    | 233990/442383 [15:22<13:16, 261.76it/s]

In [9]:
adf['datetime'] = pd.to_datetime(dts)

In [11]:
adf.head(10)

,datetime_str,geometry,icon_url,location,offense_type,url,datetime
0,"Jan. 10, 2009, 11 p.m.",POINT (-118.248596 34.059957),http://www.latimes.com/includes/projects/img/crime/crime_vehicle.png,Temple Street and Figueroa,Theft from vehicle,/crime/report/16236/,2009-01-10 23:00:00
1,"Jan. 10, 2009, 11 p.m.",POINT (-118.256068 34.042302),http://www.latimes.com/includes/projects/img/crime/crime_vehicle.png,Broadway and 9th Street,Theft from vehicle,/crime/report/16237/,2009-01-10 23:00:00
2,"Jan. 10, 2009, 11 p.m.",POINT (-118.241499 34.039904),http://www.latimes.com/includes/projects/img/crime/crime_robbery.png,800 block of East 6th Street,Robbery,/crime/report/16235/,2009-01-10 23:00:00
3,"Jan. 10, 2009, 7:10 p.m.",POINT (-118.243629 34.042139),http://www.latimes.com/includes/projects/img/crime/crime_theft.png,500 block of Crocker Street,Theft,/crime/report/16325/,2009-01-10 19:10:00
4,"Jan. 10, 2009, 6:50 p.m.",POINT (-118.264721 34.036004),http://www.latimes.com/includes/projects/img/crime/crime_robbery.png,1400 block of South Olive Street,Robbery,/crime/report/16334/,2009-01-10 18:50:00
5,"Jan. 10, 2009, 6:25 p.m.",POINT (-118.251683 34.04131),http://www.latimes.com/includes/projects/img/crime/crime_robbery.png,Santee and 8th,Robbery,/crime/report/16339/,2009-01-10 18:25:00
6,"Jan. 10, 2009, 5:30 p.m.",POINT (-118.256321 34.050419),http://www.latimes.com/includes/projects/img/crime/crime_vehicle.png,500 block of South Flower Street,Theft from vehicle,/crime/report/16358/,2009-01-10 17:30:00
7,"Jan. 10, 2009, 4:45 p.m.",POINT (-118.243364 34.052208),http://www.latimes.com/includes/projects/img/crime/crime_theft.png,100 block of West 1st Street,Theft,/crime/report/16379/,2009-01-10 16:45:00
8,"Jan. 10, 2009, 3 p.m.",POINT (-118.238051 34.045501),http://www.latimes.com/includes/projects/img/crime/crime_vehicle.png,3rd and Alameda streets,Theft from vehicle,/crime/report/16412/,2009-01-10 15:00:00
9,"Jan. 10, 2009, 2 p.m.",POINT (-118.254838 34.049115),http://www.latimes.com/includes/projects/img/crime/crime_theft.png,500 block of South Grand Avenue,Theft,/crime/report/16433/,2009-01-10 14:00:00


In [14]:
bdf = adf.drop('icon_url', axis=1)

In [22]:
bdf['year'] = bdf.datetime.dt.year
bdf['month'] = bdf.datetime.dt.month
bdf['day'] = bdf.datetime.dt.day
bdf['day_of_week'] = bdf.datetime.dt.dayofweek
bdf['hour'] = bdf.datetime.dt.hour

In [23]:
%time bdf.to_pickle('./data/DTLA_crime_data_2009_to_2019.pkl')

CPU times: user 966 ms, sys: 19.4 ms, total: 985 ms
Wall time: 990 ms
